# IDentix - Multi-Modal Biometric Preprocessing (Colab Edition)

This notebook downloads the biometric datasets directly into your Colab environment, performs feature extraction (Face embeddings & Voice MFCCs), and exports lightweight feature files.

**Why use this?**
- Save local storage space.
- Use Colab's GPU for faster face embedding extraction (optional).
- Download only small `.npy` feature files to your local IDentix project.

In [1]:
!pip install kagglehub deepface librosa numpy opencv-python tf-keras matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.0/170.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 5.1 MB/s eta 0:00:00


In [2]:
import kagglehub
import os
import numpy as np
import cv2
import librosa
from deepface import DeepFace

# 1. Download Datasets
print("Downloading datasets...")
face_dataset_path = kagglehub.dataset_download("vasukipatel/face-recognition-dataset")
voice_dataset_path = kagglehub.dataset_download("kongaevans/speaker-recognition-dataset")

print(f"Face Dataset Path: {face_dataset_path}")
print(f"Voice Dataset Path: {voice_dataset_path}")

26-02-03 23:49:25 - Directory /root/.deepface has been created
26-02-03 23:49:25 - Directory /root/.deepface/weights has been created


100%|██████████| 726M/726M [00:08<00:00, 92.5MB/s]

Extracting files...


100%|██████████| 231M/231M [00:02<00:00, 106MB/s] 

Extracting files...


Face Dataset Path: /root/.cache/kagglehub/datasets/vasukipatel/face-recognition-dataset/versions/1
Voice Dataset Path: /root/.cache/kagglehub/datasets/kongaevans/speaker-recognition-dataset/versions/1


### Feature Extraction Functions
We define the same functions used in the local IDentix project.

In [3]:
def extract_face_embeddings(image_path, model_name='VGG-Face'):
    try:
        embeddings = DeepFace.represent(
            img_path=image_path,
            model_name=model_name,
            enforce_detection=False
        )
        if embeddings and len(embeddings) > 0:
            return np.array(embeddings[0]["embedding"])
        return None
    except Exception as e:
        # print(f"Error processing {image_path}: {e}")
        return None

def extract_mfcc(audio_path, sr=16000, n_mfcc=13, duration=3):
    try:
        y, _ = librosa.load(audio_path, sr=sr)
        y_trimmed, _ = librosa.effects.trim(y)
        max_samples = sr * duration
        if len(y_trimmed) > max_samples:
            y_trimmed = y_trimmed[:max_samples]
        else:
            padding = max_samples - len(y_trimmed)
            y_trimmed = np.pad(y_trimmed, (0, padding), 'constant')

        mfccs = librosa.feature.mfcc(y=y_trimmed, sr=sr, n_mfcc=n_mfcc)
        return np.mean(mfccs.T, axis=0)
    except Exception as e:
        # print(f"Error processing {audio_path}: {e}")
        return None

In [4]:
# 2. Process Datasets and Save Features
# Note: This is a simplified loop for demonstration.
# In a real run, you would iterate through the specific dataset structure.

processed_faces = []
processed_voices = []

print("Starting processing (Sample run)... ")

# Example: Iterate through a subset of face images
face_images = []
for root, dirs, files in os.walk(face_dataset_path):
    for file in files:
        if file.endswith(('.jpg', '.png')):
            face_images.append(os.path.join(root, file))
            if len(face_images) >= 50: break # Limit for demo
    if len(face_images) >= 50: break

print(f"Extracting embeddings for {len(face_images)} face images...")
for img_path in face_images:
    emb = extract_face_embeddings(img_path)
    if emb is not None:
        processed_faces.append(emb)

# Save to .npy
np.save('face_embeddings.npy', np.array(processed_faces))
print("Saved face_embeddings.npy")

# Example: Iterate through a subset of audio files
voice_files = []
for root, dirs, files in os.walk(voice_dataset_path):
    for file in files:
        if file.endswith('.wav'):
            voice_files.append(os.path.join(root, file))
            if len(voice_files) >= 50: break
    if len(voice_files) >= 50: break

print(f"Extracting MFCCs for {len(voice_files)} audio files...")
for aud_path in voice_files:
    mfcc = extract_mfcc(aud_path)
    if mfcc is not None:
        processed_voices.append(mfcc)

# Save to .npy
np.save('voice_mfccs.npy', np.array(processed_voices))
print("Saved voice_mfccs.npy")

Starting processing (Sample run)... 
Extracting embeddings for 50 face images...
26-02-03 23:49:53 - 🔗 vgg_face_weights.h5 will be downloaded from https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5 to /root/.deepface/weights/vgg_face_weights.h5...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [00:05<00:00, 108MB/s]


Saved face_embeddings.npy
Extracting MFCCs for 50 audio files...
Saved voice_mfccs.npy


### Download Results
Run the cell below to download the processed feature files to your local machine.

In [5]:
from google.colab import files
files.download('face_embeddings.npy')
files.download('voice_mfccs.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>